# Example usage for "movekit"

In [1]:
import movekit as mkit

### Read in CSV file

In [2]:
# Enter path to CSV file
path = "./datasets/fish-5.csv"

In [3]:
# Read in CSV file using appropriate method-
data = mkit.read_data(path)
data.head()

,time,animal_id,x,y
0,1,312,405.29,417.76
1,1,511,369.99,428.78
2,1,607,390.33,405.89
3,1,811,445.15,411.94
4,1,905,366.06,451.76


#### Preprocess the data

In [4]:
preprocessed_data = mkit.preprocess(data)


 Number of missing values = y            0
x            0
animal_id    0
time         0
dtype: int64



#### Extract features

In [7]:
data_features = mkit.extract_features(preprocessed_data, fps = 10)
data_features.head(20)

,time,animal_id,x,y,distance,average_speed,average_acceleration,direction,stopped
0,1,312,405.29,417.76,0.000000,0.000000,0.000000,0.000000,0
1,2,312,405.31,417.37,0.390512,0.000000,0.000000,-87.064327,0
2,3,312,405.31,417.07,0.300000,0.000000,0.000000,-90.000000,0
3,4,312,405.30,416.86,0.210238,0.000000,0.000000,-92.726311,0
4,5,312,405.29,416.71,0.150333,0.000000,0.000000,-93.814075,0
5,6,312,405.27,416.61,0.101980,0.000000,0.000000,-101.309932,0
6,7,312,405.27,416.54,0.070000,0.000000,0.000000,-90.000000,0
7,8,312,405.27,416.49,0.050000,0.000000,0.000000,-90.000000,0
8,9,312,405.31,416.37,0.126491,0.000000,0.000000,-71.565051,0
9,10,312,405.38,416.27,0.122066,0.152162,0.000000,-55.007980,1


### Save to CSV

In [6]:
# save features to csv 
data_features.to_csv("fish-5-features.csv")